# Inserimento manuale dei dati nel database
### Matter Of Design

In [19]:
pip install mysql-connector-python boto3


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
# Librerie necessarie
import mysql.connector
import boto3
from tkinter import *
from tkinter import Tk, Label, Entry, Button, messagebox

## Inserimento dati

In [21]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

# Funzione per caricare un file su Amazon S3
def upload_to_s3(file_path, bucket_name, s3_key):
    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_path, bucket_name, s3_key)
        return f"https://{bucket_name}.s3.amazonaws.com/{s3_key}"
    except Exception as e:
        print(f"Error uploading file: {e}")
        return None

# Funzione per inserire i dati nel database
def insert_data(codice_articolo, brand, finitura, posa, collezione, dimensione, 
                prezzo_unitario, prezzo_mq, colore, spessore, tipologia_id, 
                documento_path, immagine_path, nota, peso):
    cursor = db_connection.cursor()
    
    # Caricamento del documento e immagine su S3 (se forniti)
    documento_url = upload_to_s3(documento_path, 'matterofdesign-files', f'documenti/{codice_articolo}_doc') if documento_path else None
    immagine_url = upload_to_s3(immagine_path, 'matterofdesign-files', f'immagini/{codice_articolo}_img') if immagine_path else None

    query = """
    INSERT INTO Prodotto (codice_articolo, brand, finitura, posa, collezione, 
    dimensione, prezzo_unitario, prezzo_mq, colore, spessore, 
    tipologia_id, documento, immagine, nota, peso) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (codice_articolo, brand, finitura, 
              posa if posa else None,
              collezione, dimensione, 
              prezzo_unitario if prezzo_unitario is not None else None, 
              prezzo_mq if prezzo_mq is not None else None, 
              colore, 
              spessore if spessore is not None else None, 
              tipologia_id, 
              documento_url, immagine_url, 
              nota if nota else None, 
              peso if peso is not None else None)
    cursor.execute(query, values)
    db_connection.commit()

# Funzione per raccogliere i dati dall'interfaccia e inserirli
def submit_data():
    try:
        codice_articolo = codice_articolo_entry.get()
        brand = brand_entry.get()
        finitura = finitura_entry.get()
        posa = posa_entry.get()
        collezione = collezione_entry.get()
        dimensione = dimensione_entry.get()
        prezzo_unitario = prezzo_unitario_entry.get()
        prezzo_mq = prezzo_mq_entry.get()
        prezzo_unitario = float(prezzo_unitario) if prezzo_unitario else None
        prezzo_mq = float(prezzo_mq) if prezzo_mq else None
        colore = colore_entry.get()
        spessore = spessore_entry.get()
        spessore = float(spessore) if spessore else None
        tipologia_id = int(tipologia_id_entry.get())
        documento_path = documento_entry.get()  # Percorso del file documento
        immagine_path = immagine_entry.get()    # Percorso del file immagine
        nota = nota_entry.get()
        peso = peso_entry.get()
        peso = float(peso) if peso else None

        insert_data(codice_articolo, brand, finitura, posa, collezione, dimensione, 
                    prezzo_unitario, prezzo_mq, colore, spessore, tipologia_id, 
                    documento_path, immagine_path, nota, peso)
        messagebox.showinfo("Success", "Dati inseriti correttamente!")
    except Exception as e:
        messagebox.showerror("Error", f"Errore nell'inserimento dei dati: {e}")

# Connessione al database
db_connection = connect_to_db()

# Creazione dell'interfaccia grafica
root = Tk()
root.title("Inserimento Dati Prodotto")

# Creazione dei campi di input
Label(root, text="Codice Articolo").grid(row=0)
Label(root, text="Brand").grid(row=1)
Label(root, text="Finitura").grid(row=2)
Label(root, text="Posa").grid(row=3)
Label(root, text="Collezione").grid(row=4)
Label(root, text="Dimensione").grid(row=5)
Label(root, text="Prezzo Unitario").grid(row=6)
Label(root, text="Prezzo per MQ").grid(row=7)
Label(root, text="Colore").grid(row=8)
Label(root, text="Spessore").grid(row=9)
Label(root, text="Tipologia ID").grid(row=10)
Label(root, text="Percorso Documento").grid(row=11)
Label(root, text="Percorso Immagine").grid(row=12)
Label(root, text="Nota").grid(row=13)
Label(root, text="Peso").grid(row=14)

# Creazione degli Entry per ogni campo
codice_articolo_entry = Entry(root)
brand_entry = Entry(root)
finitura_entry = Entry(root)
posa_entry = Entry(root)
collezione_entry = Entry(root)
dimensione_entry = Entry(root)
prezzo_unitario_entry = Entry(root)
prezzo_mq_entry = Entry(root)
colore_entry = Entry(root)
spessore_entry = Entry(root)
tipologia_id_entry = Entry(root)
documento_entry = Entry(root)
immagine_entry = Entry(root)
nota_entry = Entry(root)
peso_entry = Entry(root)

# Posizionamento degli Entry nella finestra
codice_articolo_entry.grid(row=0, column=1)
brand_entry.grid(row=1, column=1)
finitura_entry.grid(row=2, column=1)
posa_entry.grid(row=3, column=1)
collezione_entry.grid(row=4, column=1)
dimensione_entry.grid(row=5, column=1)
prezzo_unitario_entry.grid(row=6, column=1)
prezzo_mq_entry.grid(row=7, column=1)
colore_entry.grid(row=8, column=1)
spessore_entry.grid(row=9, column=1)
tipologia_id_entry.grid(row=10, column=1)
documento_entry.grid(row=11, column=1)
immagine_entry.grid(row=12, column=1)
nota_entry.grid(row=13, column=1)
peso_entry.grid(row=14, column=1)

# Pulsante per inviare i dati
Button(root, text="Inserisci", command=submit_data).grid(row=15, column=1)

# Avvio dell'interfaccia
root.mainloop()



Error: 2003 (HY000): Can't connect to MySQL server on 'matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com:3306' (10060)


## Correzione dati

In [22]:
# Funzione per aggiornare i dati di un prodotto
def update_product_gui():
    update_window = Toplevel(root)
    update_window.title("Aggiorna Dati Prodotto")

    # Campi di input per il codice_articolo e gli altri dati
    Label(update_window, text="Codice Articolo").grid(row=0, column=0)
    codice_articolo_entry = Entry(update_window)
    codice_articolo_entry.grid(row=0, column=1)

    def load_data():
        codice_articolo = codice_articolo_entry.get()
        cursor = db_connection.cursor()
        cursor.execute("SELECT * FROM Prodotto WHERE codice_articolo = %s", (codice_articolo,))
        result = cursor.fetchone()

        if result:
            # Mostra i valori attuali nei campi di input
            brand_entry.insert(0, result[1])
            finitura_entry.insert(0, result[2])
            posa_entry.insert(0, result[3])
            collezione_entry.insert(0, result[4])
            dimensione_entry.insert(0, result[5])
            prezzo_unitario_entry.insert(0, result[6])
            prezzo_mq_entry.insert(0, result[7])
            colore_entry.insert(0, result[8])
            spessore_entry.insert(0, result[9])
            tipologia_id_entry.insert(0, result[10])
            nota_entry.insert(0, result[13])
            peso_entry.insert(0, result[14])
        else:
            messagebox.showerror("Errore", "Prodotto non trovato!")

    def submit_update():
        codice_articolo = codice_articolo_entry.get()
        values = {
            "brand": brand_entry.get(),
            "finitura": finitura_entry.get(),
            "posa": posa_entry.get(),
            "collezione": collezione_entry.get(),
            "dimensione": dimensione_entry.get(),
            "prezzo_unitario": prezzo_unitario_entry.get() or None,
            "prezzo_mq": prezzo_mq_entry.get() or None,
            "colore": colore_entry.get(),
            "spessore": spessore_entry.get() or None,
            "tipologia_id": tipologia_id_entry.get(),
            "nota": nota_entry.get(),
            "peso": peso_entry.get() or None
        }
        
        # Genera query dinamica
        fields = ", ".join(f"{k} = %s" for k, v in values.items() if v is not None)
        query = f"UPDATE Prodotto SET {fields} WHERE codice_articolo = %s"
        params = tuple(v for v in values.values() if v is not None) + (codice_articolo,)

        try:
            cursor = db_connection.cursor()
            cursor.execute(query, params)
            db_connection.commit()
            messagebox.showinfo("Successo", "Dati aggiornati con successo!")
            update_window.destroy()
        except Exception as e:
            messagebox.showerror("Errore", f"Errore nell'aggiornamento: {e}")

    # Pulsante per caricare i dati esistenti
    Button(update_window, text="Carica Dati", command=load_data).grid(row=0, column=2)

    # Campi di input per gli altri dati
    labels = ["Brand", "Finitura", "Posa", "Collezione", "Dimensione", "Prezzo Unitario", "Prezzo per MQ",
              "Colore", "Spessore", "Tipologia ID", "Nota", "Peso"]
    entries = []

    for i, label in enumerate(labels, start=1):
        Label(update_window, text=label).grid(row=i, column=0)
        entry = Entry(update_window)
        entry.grid(row=i, column=1)
        entries.append(entry)

    # Associa gli entry ai rispettivi nomi di variabile
    (brand_entry, finitura_entry, posa_entry, collezione_entry, dimensione_entry,
     prezzo_unitario_entry, prezzo_mq_entry, colore_entry, spessore_entry,
     tipologia_id_entry, nota_entry, peso_entry) = entries

    # Pulsante per inviare i dati aggiornati
    Button(update_window, text="Aggiorna", command=submit_update).grid(row=len(labels) + 1, column=1)

# Connessione al database
db_connection = connect_to_db()

# Creazione dell'interfaccia principale
root = Tk()
root.title("Gestione Prodotto")

# Pulsante per aprire la finestra di aggiornamento
Button(root, text="Aggiorna Prodotto", command=update_product_gui).pack()

# Avvio dell'interfaccia principale
root.mainloop()

Error: 2003 (HY000): Can't connect to MySQL server on 'matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com:3306' (10060)


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\emmal\scoop\apps\python\3.12.3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\emmal\AppData\Local\Temp\ipykernel_13084\4215370631.py", line 13, in load_data
    cursor = db_connection.cursor()
             ^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'cursor'


## Verifica dati

In [23]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

def fetch_data():
    connection = connect_to_db()
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Prodotto")
    results = cursor.fetchall()
    for row in results:
        print(row)

# Esegui la funzione per vedere i dati
fetch_data()

Error: 2003 (HY000): Can't connect to MySQL server on 'matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com:3306' (10060)


AttributeError: 'NoneType' object has no attribute 'cursor'